<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Classification Metrics: ROC and AUC
              
</p>
</div>

Data Science Cohort Live NYC Feb 2023
<p>Phase 3: Topic 25</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
    

# Objectives
- Calculate and interpret probability estimates
- Adjust the threshold of a logistic regression model
- Visualize, calculate and interpret the AUC-ROC metric

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report # plot_confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.preprocessing import StandardScaler

#### What is the issue?

- Related to (basically the same as) concerns around Neyman-Pearson testing

<img src = "Images/neyman_small.png" width = 300 />

- $\alpha$ is the false positive rate (reject null when null is true)
- $\beta$ is false negative rate (accept null when null is false)

- Dependent on $t_{crit}$ OR:
    - the significance level $\alpha$
    - the probability **threshold**

#### Obvious connection
- Hypothesis testing
- Probabilistic classification 

$\alpha$ and $\beta$ depends on:
- significance level
- the structure of the hypothesis test (distribution, type of test, etc.)
- the data (sample size, etc.)

Precision, recall, F-score depends on:
- threshold
- structure of model (type of model, hyperparameters)
- the data

#### Understanding model quality

Want to systematically understand:
- how changing threshold affects:
    - true positive/false positive rate
    - precision/recall 

A nice applet. Let' play with it:

http://arogozhnikov.github.io/2015/10/05/roc-curve.html

Tuning the tolerance (the significance level):
- Traces out a curve in (true positive rate, false positive rate) space.

**Reciever Operator Characteristic**

<img src = "Images/roc_curve.png" />

Reciever operator characteristic (ROC) curve:
- Name comes from early days of radar detection.
- WW2 operators detecting enemy airplanes.

<center><img src = "Images/azm_zero.jpeg" width = 500/></center>
<center> Detecting the Japanese AZM Zero Fighter </center>


#### The ROC curve in scikit-learn

In [ ]:
from sklearn.metrics import roc_curve,RocCurveDisplay
#from sklearn.metrics import plot_roc_curve #depreciated


Load in the heart disease dataset.
[this UCI dataset](https://archive.ics.uci.edu/ml/datasets/Heart+Disease) 

In [ ]:
hd_data = pd.read_csv('Data/heart.csv')
hd_data.info()

In [ ]:
# Separate data into feature and target DataFrames
hd_X = hd_data.drop('target', axis=1)
hd_y = hd_data['target']
hd_X.head()

In [ ]:
hd_y

In [ ]:
hd_y.value_counts() # 1 = heart disease

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(hd_X, hd_y, test_size=.25,
                                                   random_state=1)
# Scale the data for modeling
hd_scaler = StandardScaler()
hd_scaler.fit(X_train)
X_train_sc = hd_scaler.transform(X_train)
X_test_sc = hd_scaler.transform(X_test)

# Train a logistic regresssion model with the train data
hd_model = LogisticRegression(random_state=42)
hd_model.fit(X_train_sc, y_train)

In [ ]:
y_pred = hd_model.predict(X_test_sc)
y_pred[:10]

When we run the `.predict()` method, `sklearn` gives us the predicted values for each transaction in our test set: 0 if predicting "no heart disease", 1 if predicting "heart disease"

Scikit-learn assumes a probability threshold of 0.5 on binary classification.

<center><img src = "Images/sigmoid.png" /></center>

Reminder: the logistic regression model doesn't actually generate predicted values of 0 or 1. It creates an S-shaped curve to approximate the data, estimating the _probability_ that they belong to the target class. This probability takes a value _between_ 0 and 1.

#### The underlying predicted probability of each class given data observation
- the .predict_proba() function

In [ ]:
y_prob = hd_model.predict_proba(X_test_sc)
y_prob[:5]

In [ ]:
y_pred[:5]

Get a 2D array:
- [P(class 0|x), P(class 1|x)] for each x in test set.

In principle:
- Can change threshold cutoff to assign to given class
- Track changes in metrics
    - True positive/false positive rate
    - Precision/recall

#### roc_curve(y_true, y_proba)
- first argument: test values
- second argument: probability of positive class    

- list of false positive rate (fpr)
- list of true positive rate (tpr)
- list of "thresholds" each fpr, tpr was calculated at:
    - actually values of decision function

### True Positive Rate
True Positive Rate (TPR) is the same as recall, measuring how many of the positive cases we correctly classified as positive.

**True Positive Rate (TPR)** = **Recall** = $\frac{TP}{TP + FN}$

Rate of correctly rejecting null (statistical power)


### False Positive Rate
False Positive Rate (FPR) measures how many of the negative casses we incorrectly classified as positive.

**False Positive Rate (TPR)** = $\frac{FP}{FP + TN}$

Rate of falsely rejecting null (Type I error)

Given test/validation set:
- calculate these metrics

#### roc_curve(y_true, y_proba)
- first argument: test values
- second argument: probability of positive class  

- list of false positive rate (fpr)
- list of true positive rate (tpr)
- list of thresholds each fpr, tpr was calculated at:

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_prob[:,1])

List of thresholds:
- in order of: no positive identifications to always identify as positive

In [ ]:
thresholds[1::]

In [ ]:
thresh_df = pd.DataFrame({'threshold': thresholds,
                          'tpr':  tpr, 'fpr': fpr}).iloc[1::, :]
thresh_df.head()

In [ ]:
fig, ax = plt.subplots()
thresh_df.plot(x = 'threshold', y = 'tpr', ax = ax)
thresh_df.plot(x = 'threshold', y = 'fpr', ax = ax)
ax.set_ylabel('True positive rate')

When threshold too high:
- Both FPR and TPR close to 0
- Never detects positive class.

When threshold gets lower:
- may be increasing true positive rate
- also increasing the rate of false positives

Common to plot true positive rate vs. false positive rate:

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (9,4))
thresh_df.plot(x = 'threshold', y = 'tpr', ax = ax[0])
thresh_df.plot(x = 'threshold', y = 'fpr', ax = ax[0])
thresh_df.plot(x = 'fpr', y = 'tpr', ax = ax[1], label = 'ROC')
ax[1].set_ylabel('True positive rate')
ax[1].set_xlabel('False positive rate')
plt.tight_layout()

#### ROC curve
- can be used to assess model quality: understand model behavior as a function of threshold
- **for a given trained model and data: get threshold sweetspot**

- for a given trained model and data: get threshold sweetspot
    - high true positive rate (good statistical power)
    - as low a false positive rate as possible (low type I error)

A useful command here: 
- directly plot ROC curve
- input our trained heart disease model and test data

In [ ]:
#plot_roc_curve(hd_model, X_test_sc, y_test);
RocCurveDisplay.from_estimator(hd_model, X_test_sc, y_test);

Lowering detection threshold:
-  If I raise significance level, lower detection threshold:
    - get more true positives
    - also get more false negatives
- Extreme case:
    - detects everything as positive class

This can be used by practicioner:
- to visually decide where to operate model threshold

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (9,4))
thresh_df.plot(x = 'threshold', y = 'tpr', ax = ax[0])
thresh_df.plot(x = 'threshold', y = 'fpr', ax = ax[0])
thresh_df.plot(x = 'fpr', y = 'tpr', ax = ax[1], label = 'ROC')
ax[1].set_ylabel('True positive rate')
ax[1].set_xlabel('False positive rate')
plt.tight_layout()

Can do this visually or can use:
- calculated TPR
- calculated FPR

Choose optimal threshold that maximizes:
    
$$ J = TPR - FPR $$

Want as many true positive identifcations while minimizing false positives 

**Known as Youden's J-statistic**

In [ ]:
thresh_df['J_stat'] = \
thresh_df['tpr'] - thresh_df['fpr']
thresh_df.head()

In [ ]:
thresh_df.plot(x = 'threshold', y= 'J_stat');

Select threshold with highest J-statistic

In [ ]:
max_selector = thresh_df.index == thresh_df['J_stat'].idxmax()

optimal_thresh = thresh_df[max_selector]
optimal_thresh

In [ ]:
fig, ax = plt.subplots()
thresh_df.plot(x = 'fpr', y = 'tpr', ax = ax, label = 'ROC')
optimal_thresh.plot.scatter(x = 'fpr', y = 'tpr', c ='r', s = 100, ax = ax, label = 'optimal' )
plt.show()

Make prediction at different threshold:
- filter on threshold value other than p = 0.5

In [ ]:
thresh = optimal_thresh['threshold'].values
# yes...this is the way to do it for binary class.
y_pred_with_threshold = (y_prob[:,1] >= thresh).astype(int)
y_pred_with_threshold

Original prediction at threshold p = 0.5:

In [ ]:
y_pred

Takeaway so far:
- ROC can be used to visually determine best threshold to operate model at
- Yuden's statistic can help with this.

#### ROC curve
- **can be used to assess model quality: understand model behavior as a function of threshold**
- for a given trained model and data: get threshold sweetspot

What does this have to do with model quality?
- Hope is that as we change threshold we create models that typically have:
    - higher TPR vs. FPRs

Good model vs bad models

<center><img src = "Images/Roc_curves_better.png" /></center>

What affects the ROC curve?

That applet again.

http://arogozhnikov.github.io/2015/10/05/roc-curve.html

Making good models are reflected in ROC curve structure:

- Use good distribution/function for data modeling
    - Model selection (logistic regression, tree model, etc.)
    - Feature Engineering

- Decrease model variance
    - Regularization
    - Get more data

Gives our ROC curve more downward L-shaped

# Oversampling

What do you do if your model doesn't perform well due to class imbalance? One of the most effective strategies is to **oversample the minority class**. That is, I give myself more data points than I really have. I could achieve this either by [bootstrapping](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) or by generating some data that is fake but close to actual data. The latter is the idea behind [SMOTE](https://imbalanced-learn.org/stable/over_sampling.html).

#### ROC-AUC score

Gets the area under the ROC curve (AUC):
- Measure of model and/or data quality
- Bad model: AUC ~ 0.5 (area of triangle)
- Good model: AUC $\rightarrow$ 1

> Remember: If my test data comprises 90% positives and only 10% negatives, then a simple classifier that always predicts "positive" will be 90% accurate! And so that would be the baseline level for a classifier on that data.

<center><img src = "images/auc.png" width = 300 /></center>

<center><img src = "Images/Roc_curves_better.png" /></center>

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
# Extract the probabilitiy predictions for the "1" class (heart disease)
y_hat_hd = y_prob[:, 1]

roc_auc_score(y_test, y_hat_hd)

Very often:
- validation tuning is done using the ROC-AUC score as metric
- tune hyperparameters to get model class with best discriminatory power

$ k = 10$ fold cross validation:
- use scaled train set for training/validation fold
- scoring on the test set is now the ROC-AUC score

In [ ]:
k = 10
C_list = [1e-3, 1e-2, 1e-1, 1, 10, 100, 1e3, 1e4]
k_list = np.arange(k)
cv_scores = []

for c in C_list :
    logreg = LogisticRegression(C = c)
    cv_loop_results = cross_validate(
                X=X_train_sc, 
                y=y_train,
                estimator=logreg, 
                cv=k,
                scoring=('roc_auc')) #the scoring is the roc auc
    cv_scores.append(dict(zip(k_list,cv_loop_results['test_score'])))
    
cv_score_df = pd.DataFrame(cv_scores) 
cv_score_df['C'] = C_list
cv_score_df.set_index('C', inplace = True)

In [ ]:
cv_score_df

In [ ]:
# mean roc auc score
cv_score_df.mean(axis = 1)

Select $C = 0.1$ as best regularization:
- based on ROC-AUC score

Take data with best discriminatory power: determined by ROC-AUC in validation
- train on full training set

In [ ]:
logreg_best = LogisticRegression(C = 0.1)
logreg_best.fit(X_train_sc, y_train)

Fitted the model with best discriminatory power:
- now should operate machine at best threshold
- ROC/AUC visualization and Youden's J statistic

- Get the predicted probabilities
- predicted class labels at threshold = 0.5

In [ ]:
y_pred_probs = logreg_best.predict_proba(X_test_sc)
y_pred_probs[0:5] # print first 5

In [ ]:
y_pred = logreg_best.predict(X_test_sc)
y_pred

Evaluate TPR, FPR vs threshold for class 1 detection:
- use roc_curve() command

In [ ]:
fpr_best, tpr_best, thresholds_best = roc_curve(y_test, y_pred_probs[:,1])
bestmod_thresh_df = pd.DataFrame({'threshold': thresholds_best,
                          'tpr':  tpr_best, 'fpr': fpr_best, 'J_stat': tpr_best - fpr_best}).iloc[1::, :]
bestmod_thresh_df.head()

Visual inspect ROC and Yuden's J maximization:
- get best threshold to operate at

With the default parameter C = 1

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (9,4))
bestmod_thresh_df.plot(x = 'threshold', y = 'tpr', ax = ax[0])
bestmod_thresh_df.plot(x = 'threshold', y = 'fpr', ax = ax[0])
bestmod_thresh_df.plot(x = 'fpr', y = 'tpr', ax = ax[1], label = 'ROC')
ax[1].set_ylabel('True positive rate')
ax[1].set_xlabel('False positive rate')
plt.tight_layout()

In [ ]:
# with default log regression
fig, ax = plt.subplots(1,2, figsize = (9,4))
thresh_df.plot(x = 'threshold', y = 'tpr', ax = ax[0])
thresh_df.plot(x = 'threshold', y = 'fpr', ax = ax[0])
thresh_df.plot(x = 'fpr', y = 'tpr', ax = ax[1], label = 'ROC')
ax[1].set_ylabel('True positive rate')
ax[1].set_xlabel('False positive rate')
plt.tight_layout()

Tuning and optimizing on ROC/AUC makes a difference:
- region where we can decrease threshold (making more sensitive detector)
- no increase in FPR, increasing TPR

Finding the best threshold value operating point:
- using Yuden's J

But ultimately this is up to you. Can assess visually.

In [ ]:
best_idx = bestmod_thresh_df['J_stat'].idxmax()
best_point = pd.DataFrame(bestmod_thresh_df.iloc[best_idx]).T
best_point

Using Youden's J-statistic:

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (9,4))
bestmod_thresh_df.plot(x = 'threshold', y = 'tpr', ax = ax[0])
bestmod_thresh_df.plot(x = 'threshold', y = 'fpr', ax = ax[0])
ax[0].axvline(best_point['threshold'].values, c = 'r', linestyle = '--')

bestmod_thresh_df.plot(x = 'fpr', y = 'tpr', ax = ax[1], label = 'ROC')
best_point.plot.scatter(x = 'fpr', y = 'tpr', ax = ax[1], c ='r', s = 100, label = 'Optimal')
ax[1].set_ylabel('True positive rate')
ax[1].set_xlabel('False positive rate')

plt.tight_layout()

Is this optimal? Maybe, maybe not.
- If we care about keeping fpr low, choose threshold = 0.4 instead.

Once satisfied with operating threshold:
- filter probabilities according to threshold

In [ ]:
#filter on class 1 probabilities
y_pred_best_with_threshold = (y_pred_probs[:,1] >= 0.4).astype(int)
y_pred_best_with_threshold

Let's get classification report for this AUC-selected, threshold tuned model:
- Threshold at 0.4

In [ ]:
print(classification_report(y_test, y_pred_best_with_threshold))

Look at confusion matrix at this threshold:

In [ ]:
conf_mat_best = confusion_matrix(y_test, y_pred_best_with_threshold)
fig, ax = plt.subplots()
sns.heatmap(conf_mat_best, annot = True, ax = ax)
ax.set_ylabel(r'$y_{true}$', size = 15)
ax.set_xlabel(r'$y_{pred}$', size = 15)
plt.show()

Without any feature engineering, this is a good job.


Instructive to look at original model performance vs. tuned/thresholded model.

In [ ]:
print(classification_report(y_test, y_pred_best_with_threshold))

In [ ]:
y_pred_orig = hd_model.predict(X_test_sc)
print(classification_report(y_test, y_pred_orig))

These kinds of considerations:
- ROC/AUC and model discriminatory power
- Tuning thresholds

Can be extended to multi-class problems:
- one verse rest (OvR)
- one verse one (OvO)

Custom code needs to be built and time consuming